# Convolutional Neural Networks

## Project: Write an Algorithm for Landmark Classification

### A simple app

In this notebook we build a very simple app that uses our exported model.

> <img src="static_images/icons/noun-info-2558213.png" alt="?" style="width:25px"/> Note how we are not importing anything from our source code (we do not use any module from the ``src`` directory). This is because the exported model, differently from the model weights, is a standalone serialization of our model and therefore it does not need anything else. You can ship that file to anybody, and as long as they can import ``torch``, they will be able to use your model. This is very important for releasing pytorch models to production.

### Test your app
Go to a search engine for images (like Google Images) and search for images of some of the landmarks, like the Eiffel Tower, the Golden Gate Bridge, Machu Picchu and so on. Save a few examples locally, then upload them to your app to see how your model behaves!

The app will show the top 5 classes that the model think are most relevant for the picture you have uploaded

In [5]:
!pip install ipywidgets


Defaulting to user installation because normal site-packages is not writeable


In [6]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter nbextension install --py widgetsnbextension --sys-prefix


Enabling notebook extension jupyter-js-widgets/extension...
Traceback (most recent call last):
  File "/opt/conda/bin/jupyter-nbextension", line 11, in <module>
    sys.exit(main())
  File "/opt/conda/lib/python3.7/site-packages/jupyter_core/application.py", line 270, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.7/site-packages/notebook/nbextensions.py", line 988, in start
    super(NBExtensionApp, self).start()
  File "/opt/conda/lib/python3.7/site-packages/jupyter_core/application.py", line 259, in start
    self.subapp.start()
  File "/opt/conda/lib/python3.7/site-packages/notebook/nbextensions.py", line 896, in start
    self.toggle_nbextension_python(self.extra_args[0])
  File "/opt/conda/lib/python3.7/site-packages/notebook/nbextensions.py", line 872, in toggle_nbextension_python
 

## (optional) Standalone app or web app

You can run this notebook as a standalone app on your computer by following these steps:

1. Download this notebook in a directory on your machine
2. Download the model export (for example, ``checkpoints/transfer_exported.pt``) in a subdirectory called ``checkpoints`` within the directory where you save the app.ipynb notebook
3. Install voila if you don't have it already (``pip install voila``)
4. Run your app: ``voila app.ipynb --show_tracebacks=True``
5. Customize your notebook to make your app prettier and rerun voila

You can also deploy this app as a website using Binder: https://voila.readthedocs.io/en/stable/deploy.html#deployment-on-binder

In [4]:
from ipywidgets import VBox, Button, FileUpload, Output, Label
from PIL import Image
from IPython.display import display
import io
import numpy as np
import torch
import torchvision.transforms as T
from src.model import MyModel  # Ensure this matches your actual module and class name

# Load the model
model = MyModel(num_classes=5)  # Ensure the number of classes matches your model
model_path = "checkpoints/model.pth"
model.load_state_dict(torch.load(model_path))
model.eval()  # Ensure the model is in evaluation mode

# Define class names (update with actual class names)
class_names = ['Eiffel Tower', 'Golden Gate Bridge', 'Machu Picchu', 'Statue of Liberty', 'Sydney Opera House']
model.class_names = class_names

def preprocess_image(img):
    img = img.resize((256, 256))
    img = img.crop((16, 16, 240, 240))  # Center crop to 224x224
    preprocess = T.Compose([
        T.ToTensor(),
        T.Normalize(mean=[0.4638, 0.4725, 0.4687], std=[0.2699, 0.2706, 0.3018])
    ])
    img = preprocess(img).unsqueeze_(0)
    return img

def on_click_classify(change):
    try:
        # Load image that has been uploaded
        if not btn_upload.value:
            raise ValueError("No image uploaded")
        fn = io.BytesIO(list(btn_upload.value.values())[0]['content'])
        img = Image.open(fn)
        img.load()

        # Clear the previous output (if any)
        out_pl.clear_output()

        # Display the image
        with out_pl:
            ratio = img.size[0] / img.size[1]
            c = img.copy()
            c.thumbnail([ratio * 200, 200])
            display(c)

        # Preprocess the image
        timg = preprocess_image(img)

        # Calling the model
        with torch.no_grad():
            outputs = model(timg)
            softmax = torch.nn.functional.softmax(outputs, dim=1).cpu().numpy().squeeze()

        # Get the indexes of the classes ordered by softmax (larger first)
        idxs = np.argsort(softmax)[::-1]

        # Loop over the classes with the largest softmax
        for i in range(5):
            # Get softmax value
            p = softmax[idxs[i]]

            # Get class name
            landmark_name = model.class_names[idxs[i]]

            labels[i].value = f"{landmark_name} (prob: {p:.2f})"
    except Exception as e:
        out_pl.clear_output()
        with out_pl:
            display(f"Error: {str(e)}")

# Button for file upload
btn_upload = FileUpload(accept='image/*', multiple=False)

# Button to run the classification
btn_run = Button(description="Classify")
btn_run.on_click(on_click_classify)

# Button to clear output
btn_clear = Button(description="Clear")
def on_click_clear(change):
    out_pl.clear_output()
    for label in labels:
        label.value = ""
btn_clear.on_click(on_click_clear)

# Labels to display the top 5 predictions
labels = [Label() for _ in range(5)]

# Output widget for displaying the image
out_pl = Output()
out_pl.clear_output()

# Arrange widgets in a vertical box
wgs = [Label("Please upload a picture of a landmark"), btn_upload, btn_run, btn_clear, out_pl] + labels
display(VBox(wgs))


# Create your submission archive

Now that you are done with your project, please run the following cell. It will generate a file containing all the code you have written, as well as the notebooks. Please submit that file to complete your project

In [1]:
!python src/create_submit_pkg.py

executing: jupyter nbconvert --to html cnn_from_scratch.ipynb
[NbConvertApp] Converting notebook cnn_from_scratch.ipynb to html
[NbConvertApp] Writing 967288 bytes to cnn_from_scratch.html
executing: jupyter nbconvert --to html app.ipynb
[NbConvertApp] Converting notebook app.ipynb to html
[NbConvertApp] Writing 300910 bytes to app.html
executing: jupyter nbconvert --to html transfer_learning.ipynb
[NbConvertApp] Converting notebook transfer_learning.ipynb to html
[NbConvertApp] Writing 413302 bytes to transfer_learning.html
Adding files to submission_2024-05-28T18h20m.tar.gz
src/predictor.py
src/model.py
src/__init__.py
src/helpers.py
src/transfer.py
src/train.py
src/data.py
src/optimization.py
src/create_submit_pkg.py
cnn_from_scratch.ipynb
app.ipynb
transfer_learning.ipynb
app.html
transfer_learning.html
cnn_from_scratch.html

----------------------------------------------------------------
Done. Please submit the file submission_2024-05-28T18h20m.tar.gz
----------------------------